## Function Calling

The latest versions of gpt-35-turbo and gpt-4 have been fine-tuned to work with functions and are able to both determine when and how a function should be called.

If one or more functions are included in your request, the model will then determine if any of the functions should be called based on the context of the prompt.

When the model determines that a function should be called, it will then respond with a JSON object including the arguments for the function.

https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/function-calling

In [2]:
import openai
import json
import os
from IPython.display import display, HTML, JSON, Markdown
from dotenv import load_dotenv

In [3]:
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_DEPLOYMENT_ENDPOINT = os.getenv("OPENAI_DEPLOYMENT_ENDPOINT")

# Configure OpenAI API
openai.api_type = "azure"
# The API version required for Function Calling
openai.api_version = "2023-07-01-preview"
openai.api_base = OPENAI_DEPLOYMENT_ENDPOINT
openai.api_key = OPENAI_API_KEY

#### Function definition for the model

In [4]:
def get_function_call(messages, function_call="auto"):
    # Define the functions to use
    functions = [
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
    ]

    # Call the model with the user query (messages) and the functions defined in the functions parameter
    response = openai.ChatCompletion.create(
        deployment_id="gpt-35-turbo-0613",
        messages=messages,
        functions=functions, #Pass list of functions to be called
        function_call=function_call,
    )

    return response

In [5]:
#User asks the system about the weather in plain English
message = [
    {"role": "user", "content": "What's the weather like in San Francisco now?"}]

#Call the model with the user query
function_to_call_response = get_function_call(message, "auto")

#process output and check which function to call
function_name = function_to_call_response["choices"][0]["message"]["function_call"]["name"]
function_parameters = json.loads( function_to_call_response["choices"][0]["message"]["function_call"]["arguments"])
print (f""" \nfunction: {function_name} with parameters: {function_parameters}""")

 
function: get_current_weather with parameters: {'location': 'San Francisco, CA'}
